# Goals

## Challenge 1
For this goal, you are given a number of CSV files (2), each of which have their first row with the field name.

You goal is to create a context manager that you can use to produce the data from each file in a named tuple with field names corresponding to the  header row field names.

You should use the `csv` module's `reader` function to help with parsing the data.

Your context manager should be generic in the sense that it should just need the file name, no other configuration or hardcoded functionality is required. You do not need to worry about data types for this goal - just return every field as a string.

In addition, your context manager should produce lazy iterators.

Implement this using a class that implements the context manager protocol

## Challenge 2
The goal is to reproduce the work you did in Goal 1, but using a generator function and the `contextlib` `contextmanager` decorator.

In [7]:
from collections import namedtuple
import csv

In [120]:
class CSVDataIterator:
    ''' Reads the CSV file, generates a "Generator" where data is parses in named tuple
    '''
    def __init__(self, file_name: str, mode: str, separator: str):
        ''' Initializes the class with properties required for reading the file
        
        Parameters:
        ----------
        file_name:  File Name in STRING, which to be read
        mode:       'r' for read, 'w' for write and 'a' for append. Although only 'r' is supported
        separator:  Seperator which seperates the data in each row. Generally it will be ',' 
                    but it can be any charecter
        
        '''
        if mode != 'r':
            raise ValueError("Only 'r' mode is supported for reading CSV files.")

        self._file = file_name
        self._mode = mode
        self._separator = separator
        # Header of the CSV file is saved in this list. It is assumed that only one 
        # line of header is present in the CSV file
        self._header = []

    def __iter__(self):
        ''' Iterator for the "for" loop 
        '''
        return self
    
    def __next__(self):
        ''' To fetch the next time. Each row is fetched and then new line charecter is stripped
        and each row is then split according to "seperator" which was sending during construction 
        of the object 
        '''
        try:
            row = next(self._fd).strip('\n').split(self._separator)
            data = self._csv_data(*row)
            return data
        except StopIteration:
            raise StopIteration

    def __enter__(self):
        ''' When enter the context, read the first line (which is header) and return the object
        '''
        try:
            self._fd = open(self._file, self._mode)
        except IOError as e:
            print(f"Error opening file: {e}")
            raise

        self._header = list(next(self._fd).strip('\n').split(self._separator))
        self._csv_data = namedtuple('csv_data', self._header)
        if self._fd:
            print("File opened properly")
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        ''' Upon exiting the context close the file and display any error/exception occurred. 
        if exception has occured then stop the excecution of the code
        '''
        self._fd.close()
        if (self._fd.closed):
            print("File closed properly")
        else:
            print("File has not closed properly. Check for errors")

        if exc_type:
            print (f"Excepiton {exc_type} occured \n{exc_value}")

        return False


In [122]:
with CSVDataIterator('cars-2.csv', 'r', ';') as file:
    # print(file._header)
    for row in file:
        print (row.Car)

File opened properly
Chevrolet Chevelle Malibu
Buick Skylark 320
Plymouth Satellite
AMC Rebel SST
Ford Torino
Ford Galaxie 500
Chevrolet Impala
Plymouth Fury iii
Pontiac Catalina
AMC Ambassador DPL
Citroen DS-21 Pallas
Chevrolet Chevelle Concours (sw)
Ford Torino (sw)
Plymouth Satellite (sw)
AMC Rebel SST (sw)
Dodge Challenger SE
Plymouth 'Cuda 340
Ford Mustang Boss 302
Chevrolet Monte Carlo
Buick Estate Wagon (sw)
Toyota Corolla Mark ii
Plymouth Duster
AMC Hornet
Ford Maverick
Datsun PL510
Volkswagen 1131 Deluxe Sedan
Peugeot 504
Audi 100 LS
Saab 99e
BMW 2002
AMC Gremlin
Ford F250
Chevy C20
Dodge D200
Hi 1200D
Datsun PL510
Chevrolet Vega 2300
Toyota Corolla
Ford Pinto
Volkswagen Super Beetle 117
AMC Gremlin
Plymouth Satellite Custom
Chevrolet Chevelle Malibu
Ford Torino 500
AMC Matador
Chevrolet Impala
Pontiac Catalina Brougham
Ford Galaxie 500
Plymouth Fury iii
Dodge Monaco (sw)
Ford Country Squire (sw)
Pontiac Safari (sw)
AMC Hornet Sportabout (sw)
Chevrolet Vega (sw)
Pontiac Firebi

In [123]:
with CSVDataIterator('personal_info.csv', 'r', ',') as file:
    print(file._header)
    for row in file:
        print (row.first_name)

File opened properly
['ssn', 'first_name', 'last_name', 'gender', 'language']
Sebastiano
Cayla
Nomi
Justinian
Claudianus
Federico
Angelina
Moselle
Audi
Mackenzie
Martino
Amberly
Giacopo
Jule
Casandra
Levey
Chiquita
Herold
Auroora
Lucien
Dareen
Laina
Alexi
Blondie
Eugen
Lorry
Sal
Spenser
Kare
Christiane
Florella
Quinta
Wilona
Tresa
Mellicent
Neddy
Chucho
Tobie
Jinny
Norton
Danella
Jemie
Shalom
Simone
Tilly
Tedman
Davidson
Pam
Joletta
Stirling
Donalt
Susanetta
Price
Tim
Candy
Cyrille
Barron
Margaux
Hallie
Donny
Leon
Baron
Korry
Bendicty
Janessa
Aime
Jordan
Andrey
Wheeler
Helenka
Bram
Piggy
Urbain
Portia
Mel
Annelise
Ainslee
Trey
Carol
Ardyce
Johannes
Melisse
Melina
Kiri
Vassily
Babbette
Clemence
Lester
Reyna
Dane
Teddie
Toby
Vergil
Salvidor
Phillida
Everett
Alex
Trude
Hedwiga
Rey
Sayer
Drucie
Benton
Corny
Olivero
Errick
Ferdinande
Fan
Ebba
Lavinie
Sunny
Curran
Fern
Bary
Kore
Manfred
Alexa
Shandy
Hazlett
Clifford
Belita
Arnold
Vernen
Gorden
Hadlee
Calhoun
Trev
Edvard
Jacquelynn
Talyah
Mar

In [112]:
with open('personal_info.csv', 'r') as file:
    header = (next(file)).strip('\n').split(',')
    some_var = namedtuple('some_var', header)
    print(some_var)

<class '__main__.some_var'>


In [8]:
import csv
from collections import namedtuple

with open('personal_info.csv', 'r') as file:
    header_info = next(file).strip('\n').split(',')
    csv_data = namedtuple('csv_data', header_info)
    csv_file_rd = csv.reader(file)
    print(next(csv_file_rd))
    print(next(csv_file_rd))
    print(next(csv_file_rd))
    print(next(csv_file_rd))
    # for row in csv_file_rd:
    #     data = csv_data(*row)
    #     print(data.ssn)

['100-53-9824', 'Sebastiano', 'Tester', 'Male', 'Icelandic']
['101-71-4702', 'Cayla', 'MacDonagh', 'Female', 'Lao']
['101-84-0356', 'Nomi', 'Lipprose', 'Female', 'Yiddish']
['104-22-0928', 'Justinian', 'Kunzelmann', 'Male', 'Dhivehi']
